![A soccer pitch for an international match.](soccer-pitch.jpg)

You're working as a sports journalist at a major online sports media company, specializing in soccer analysis and reporting. You've been watching both men's and women's international soccer matches for a number of years, and your gut instinct tells you that more goals are scored in women's international football matches than men's. This would make an interesting investigative article that your subscribers are bound to love, but you'll need to perform a valid statistical hypothesis test to be sure!

While scoping this project, you acknowledge that the sport has changed a lot over the years, and performances likely vary a lot depending on the tournament, so you decide to limit the data used in the analysis to only official `FIFA World Cup` matches (not including qualifiers) since `2002-01-01`.

You create two datasets containing the results of every official men's and women's international football match since the 19th century, which you scraped from a reliable online source. This data is stored in two CSV files: `women_results.csv` and `men_results.csv`.

The question you are trying to determine the answer to is:

> Are more goals scored in women's international soccer matches than men's?

You assume a **10% significance level**, and use the following null and alternative hypotheses:

$H_0$ : The mean number of goals scored in women's international soccer matches is the same as men's.

$H_A$ : The mean number of goals scored in women's international soccer matches is greater than men's.

In [42]:
# Start your code here!
import pandas as pd
df_men_matches = pd.read_csv('./men_results.csv', parse_dates=['date'])
df_women_matches = pd.read_csv('./women_results.csv', parse_dates=['date'])

In [43]:
print(df_men_matches.shape)
df_men_matches.head()

(44353, 7)


,Unnamed: 0,date,home_team,away_team,home_score,away_score,tournament
0,0,1872-11-30,Scotland,England,0,0,Friendly
1,1,1873-03-08,England,Scotland,4,2,Friendly
2,2,1874-03-07,Scotland,England,2,1,Friendly
3,3,1875-03-06,England,Scotland,2,2,Friendly
4,4,1876-03-04,Scotland,England,3,0,Friendly


In [44]:
print(df_women_matches.shape)
df_women_matches.head()

(4884, 7)


,Unnamed: 0,date,home_team,away_team,home_score,away_score,tournament
0,0,1969-11-01,Italy,France,1,0,Euro
1,1,1969-11-01,Denmark,England,4,3,Euro
2,2,1969-11-02,England,France,2,0,Euro
3,3,1969-11-02,Italy,Denmark,3,1,Euro
4,4,1975-08-25,Thailand,Australia,3,2,AFC Championship


In [49]:
df_men_fifa=\
df_men_matches[
  (df_men_matches['tournament']=='FIFA World Cup') &
  (df_men_matches['date'].dt.year>=2002)
]
print(df_men_fifa.shape)
df_men_fifa.head(2)

(384, 7)


,Unnamed: 0,date,home_team,away_team,home_score,away_score,tournament
25164,25164,2002-05-31,France,Senegal,0,1,FIFA World Cup
25165,25165,2002-06-01,Germany,Saudi Arabia,8,0,FIFA World Cup


In [50]:
df_women_fifa=\
df_women_matches[
  (df_women_matches['tournament']=='FIFA World Cup') &
  (df_women_matches['date'].dt.year>=2002)
]
print(df_women_fifa.shape)
df_women_fifa.head(2)

(200, 7)


,Unnamed: 0,date,home_team,away_team,home_score,away_score,tournament
1600,1600,2003-09-20,Nigeria,North Korea,0,3,FIFA World Cup
1601,1601,2003-09-20,Norway,France,2,0,FIFA World Cup


In [109]:
def prep_datasets(df):
  df_prep = df.loc[:, ['date', 'home_score', 'away_score']]
  df_prep.insert(0, 'year', df['date'].dt.year)
  df_prep['goals'] = df['home_score']+df['away_score']
  df_prep = df_prep.groupby(by='year')['goals'].agg(['sum', 'count', 'mean']).sort_index(ascending=False).reset_index()
  return df_prep


In [111]:
df_men_yearly_goals = prep_datasets(df_men_fifa)
df_women_yearly_goals = prep_datasets(df_women_fifa)
df_men_yearly_goals

,year,sum,count,mean
0,2022,172,64,2.687500
1,2018,169,64,2.640625
2,2014,171,64,2.671875
3,2010,145,64,2.265625
4,2006,147,64,2.296875
5,2002,161,64,2.515625


In [112]:
df_women_yearly_goals

,year,sum,count,mean
0,2019,146,52,2.807692
1,2015,146,52,2.807692
2,2011,86,32,2.687500
3,2007,111,32,3.468750
4,2003,107,32,3.343750


In [115]:
df_comp = pd.DataFrame(
{ 'men_mean_goals': list(df_men_yearly_goals.loc[1:,:]['mean']),
  'women_mean_goals':list(df_women_yearly_goals['mean'])
})
df_comp

,men_mean_goals,women_mean_goals
0,2.640625,2.807692
1,2.671875,2.807692
2,2.265625,2.687500
3,2.296875,3.468750
4,2.515625,3.343750


In [152]:
import pingouin
from scipy.stats import mannwhitneyu

In [117]:
pingouin.wilcoxon(
  x=df_comp['men_mean_goals'],
  y=df_comp['women_mean_goals']
)

,W-val,alternative,p-val,RBC,CLES
Wilcoxon,0.0,two-sided,0.0625,-1.0,0.0


In [128]:
res = pingouin.wilcoxon(
  x=df_comp['men_mean_goals'],
  y=df_comp['women_mean_goals']
)
res

,W-val,alternative,p-val,RBC,CLES
Wilcoxon,0.0,two-sided,0.0625,-1.0,0.0


In [140]:
p_val = res['p-val'].values[0]
result_dict = {
  "p_val": p_val,
  "result": 'fail to reject' if p_val >= 0.1 else 'reject'
}
result_dict

{'p_val': 0.0625, 'result': 'reject'}

In [143]:
df_men_fifa
# df_women_fifa

,Unnamed: 0,date,home_team,away_team,home_score,away_score,tournament
25164,25164,2002-05-31,France,Senegal,0,1,FIFA World Cup
25165,25165,2002-06-01,Germany,Saudi Arabia,8,0,FIFA World Cup
25166,25166,2002-06-01,Republic of Ireland,Cameroon,1,1,FIFA World Cup
25167,25167,2002-06-01,Uruguay,Denmark,1,2,FIFA World Cup
25168,25168,2002-06-02,Argentina,Nigeria,1,0,FIFA World Cup
...,...,...,...,...,...,...,...
44343,44343,2022-12-10,England,France,1,2,FIFA World Cup
44345,44345,2022-12-13,Argentina,Croatia,3,0,FIFA World Cup
44346,44346,2022-12-14,France,Morocco,2,0,FIFA World Cup
44350,44350,2022-12-17,Croatia,Morocco,2,1,FIFA World Cup


In [149]:
# Create group and goals_scored columns
df_men_fifa.loc[:,"group"] = "men"
df_women_fifa.loc[:,"group"] = "women"
df_men_fifa.loc[:,"goals_scored"] = df_men_fifa.loc[:,"home_score"] + df_men_fifa.loc[:,"away_score"]
df_women_fifa.loc[:, "goals_scored"] = df_women_fifa.loc[:,"home_score"] + df_women_fifa.loc[:,"away_score"]
df_combined = pd.concat([df_women_fifa, df_men_fifa], axis=0, ignore_index=True)
df_combined

,Unnamed: 0,date,home_team,away_team,home_score,away_score,tournament,group,goals_scored
0,1600,2003-09-20,Nigeria,North Korea,0,3,FIFA World Cup,women,3
1,1601,2003-09-20,Norway,France,2,0,FIFA World Cup,women,2
2,1602,2003-09-20,Germany,Canada,4,1,FIFA World Cup,women,5
3,1603,2003-09-20,Japan,Argentina,6,0,FIFA World Cup,women,6
4,1604,2003-09-21,United States,Sweden,3,1,FIFA World Cup,women,4
...,...,...,...,...,...,...,...,...,...
579,44343,2022-12-10,England,France,1,2,FIFA World Cup,men,3
580,44345,2022-12-13,Argentina,Croatia,3,0,FIFA World Cup,men,3
581,44346,2022-12-14,France,Morocco,2,0,FIFA World Cup,men,2
582,44350,2022-12-17,Croatia,Morocco,2,1,FIFA World Cup,men,3


In [150]:
# Transform the data for the pingouin Mann-Whitney U t-test/Wilcoxon-Mann-Whitney test
df_combined = df_combined[["goals_scored", "group"]]
df_combined_wide = df_combined.pivot(columns="group", values="goals_scored")
df_combined_wide

group,men,women
0,NaN,3.0
1,NaN,2.0
2,NaN,5.0
3,NaN,6.0
4,NaN,4.0
...,...,...
579,3.0,NaN
580,3.0,NaN
581,2.0,NaN
582,3.0,NaN


In [154]:
# Perform right-tailed Wilcoxon-Mann-Whitney test with pingouin
results_pg = pingouin.mwu(x=df_combined_wide["women"],
                          y=df_combined_wide["men"],
                          alternative="greater")

# Alternative SciPy solution: Perform right-tailed Wilcoxon-Mann-Whitney test with scipy
results_scipy = mannwhitneyu(x=df_women_fifa["goals_scored"],
                             y=df_men_fifa["goals_scored"],
                             alternative="greater")

# Extract p-value as a float
p_val = results_pg["p-val"].values[0]

# Determine hypothesis test result using sig. level
if p_val <= 0.01:
    result = "reject"
else:
    result = "fail to reject"

result_dict = {"p_val": p_val, "result": result}

In [155]:
result_dict

{'p_val': 0.005106609825443641, 'result': 'reject'}